## LCC calculation tests (for whole track)

Notebook to test the LCC calculations for the whole track, i.e., high and low rail.

Version 0.1 (2025-05-07): initial version with simple exploration of all possible maintenance strategies.


## Setting parameters

In [14]:
import pandas as pd # type: ignore
import seaborn as sns # type: ignore
import matplotlib.pyplot as plt # type: ignore

In [15]:
# Define the high and low rail profile
profile_high_rail = "MB5"  # MB5 (default) or MB6
profile_low_rail = "MB5"  # MB5 (default) or MB6

# Define the axle load (in tonnes)
load = 30

# The actual path to your Excel file
file_path = '../data/raw/raw_data_structured_with_load.csv'

## Reading input data

In [16]:
# Ensure the preprocessings module is accessible
import sys
sys.path.append('../')  # Adjust the path to the root directory containing 'preprocessings'

# Import the function to read the input data
from preprocessings.read_input_data import read_input_data

# Call the function to read the data
data_df = read_input_data(file_path)

## LCC calculation

In [19]:
import numpy as np  # type: ignore
from scipy.interpolate import PchipInterpolator  # type: ignore
from rail_analysis.rail_measures import (
    get_h_index,
    get_wear_data,
    get_rcf_residual,
    get_rcf_depth,
)

DEFAULT_PROFILE = 'MB5'  
DEFAULT_GAUGE_WIDENING = 1  


def get_annuity_track(
    data_df,
    grinding_freq_low,     # grinding interval (months) for low rail 
    grinding_freq_high,    # grinding interval (months) for high rail 
    gauge_freq,            # tamping interval (months)
    profile_low_rail=DEFAULT_PROFILE,
    profile_high_rail=DEFAULT_PROFILE,
    track_results=False,
    gauge_widening_per_year=DEFAULT_GAUGE_WIDENING
):
    """
    Calculate the annuity (LCC per year) and track lifetime for both rails.

    Returns:
      (annuity, lifetime, [optional] history)
    """

    # --- PARAMETERS ---
    DR = 0.04
    TRACK_LEN = 1000
    TECH_LIFE_Y = 15
    MAX_MONTHS = TECH_LIFE_Y * 12

    C_POSS = 50293
    C_GRIND = 50
    C_TAMP  = 40
    T_GRIND_HR = 2
    T_TAMP_HR  = 5

    H_MAX = 14
    RCF_MAX = 0.5

    TRACK_RENEW = 6500 * TRACK_LEN
    RAIL_RENEW  = 1500 * TRACK_LEN

    # --- LOAD TABLES ---
    Ht_H = get_h_index(data_df, profile=profile_high_rail)
    NW_H = get_wear_data(data_df, profile=profile_high_rail)
    RCF_RES_H = get_rcf_residual(data_df, profile=profile_high_rail)
    RCF_DEP_H = get_rcf_depth(data_df, profile=profile_high_rail)

    Ht_L = get_h_index(data_df, profile=profile_low_rail)
    NW_L = get_wear_data(data_df, profile=profile_low_rail)
    RCF_RES_L = get_rcf_residual(data_df, profile=profile_low_rail)
    RCF_DEP_L = get_rcf_depth(data_df, profile=profile_low_rail)

    gauge_levels = [1440, 1445, 1450, 1455]

    # --- STATE & ACCUMULATORS ---
    PV_maint = 0.0
    PV_renew = TRACK_RENEW
    PV_cap   = 0.0

    H_H = H_L = 0.0
    R_H = R_L = 0.0
    gauge = gauge_levels[0]

    since_grind_H = since_grind_L = 1
    since_tamp        = 1

    history = [] if track_results else None
    lifetime = TECH_LIFE_Y

    # --- SIMULATION LOOP ---
    for m in range(1, MAX_MONTHS + 1):
        t = m / 12
        gauge += gauge_widening_per_year / 12

        # --- Grinding for each rail ---
        for rail in ('H','L'):
            freq = grinding_freq_high if rail=='H' else grinding_freq_low
            since = since_grind_H if rail=='H' else since_grind_L
            H_curr = H_H if rail=='H' else H_L
            R_curr = R_H if rail=='H' else R_L
            Ht = Ht_H if rail=='H' else Ht_L
            NW = NW_H if rail=='H' else NW_L
            RRes = RCF_RES_H if rail=='H' else RCF_RES_L
            RDep = RCF_DEP_H if rail=='H' else RCF_DEP_L

            if since == freq:
                # apply grinding cost
                PV_maint += (C_GRIND*TRACK_LEN)/(1+DR)**t
                PV_cap   += (T_GRIND_HR*C_POSS)/(1+DR)**t

                # update H-index
                ΔH_g = PchipInterpolator(gauge_levels, Ht[Ht['Month']==freq]['Value'])(gauge)
                ΔN   = PchipInterpolator(gauge_levels, NW[NW['Month']==since]['Value'])(gauge)
                H_curr += ΔH_g - ΔN

                # update RCF residual
                rcf_r = PchipInterpolator(gauge_levels, RRes[RRes['Month']==freq]['Value'])(gauge)
                if rcf_r>0:
                    R_curr += rcf_r

                since = 0
            else:
                # natural wear + new RCF
                # if NW[NW['Month']==since] is empty, use the last value
                if NW[NW['Month']==since].empty:
                    tt = NW[NW['Month']==since-1].iloc[-1]
                ΔN = PchipInterpolator(gauge_levels, NW[NW['Month']==since]['Value'])(gauge)
                ΔR = PchipInterpolator(gauge_levels, RDep[RDep['Month']==since]['Value'])(gauge)
                H_curr += ΔN
                R_curr += ΔR

            # save back
            if rail=='H':
                since_grind_H = since+1
                H_H, R_H      = H_curr, R_curr
            else:
                since_grind_L = since+1
                H_L, R_L      = H_curr, R_curr

        # --- Gauge correction (tamping) ---
        if since_tamp == gauge_freq:
            PV_maint += (C_TAMP*TRACK_LEN)/(1+DR)**t
            PV_cap   += (T_TAMP_HR*C_POSS)/(1+DR)**t
            gauge     = gauge_levels[0]
            since_tamp= 0
        since_tamp += 1

        # --- Double grind if RCF excess ---
        for rail, since_attr in (('H', 'since_grind_H'), ('L','since_grind_L')):
            R_curr = R_H if rail=='H' else R_L
            H_curr = H_H if rail=='H' else H_L
            since   = locals()[since_attr]

            if R_curr >= RCF_MAX:
                # double grind
                PV_maint += (5/3*C_GRIND*TRACK_LEN)/(1+DR)**t
                PV_cap   += (5/3*T_GRIND_HR*C_POSS)/(1+DR)**t

                ΔH1 = PchipInterpolator(gauge_levels, Ht_H[Ht_H['Month']==since+1]['Value'])(gauge)
                ΔH2 = PchipInterpolator(gauge_levels, Ht_H[Ht_H['Month']==1]['Value'])(gauge)
                H_curr += ΔH1 + ΔH2
                R_curr  = 1
                since   = 1

            # save back
            if rail=='H':
                H_H, R_H = H_curr, R_curr
                since_grind_H = since
            else:
                H_L, R_L = H_curr, R_curr
                since_grind_L = since

        # --- Rail renewal if worn out ---
        for H_curr, R_curr, name in ((H_H,R_H,'H'),(H_L,R_L,'L')):
            if H_curr > H_MAX:
                PV_renew += RAIL_RENEW/(1+DR)**t
                if name=='H': H_H, R_H = 0,0
                else:         H_L, R_L = 0,0

        # --- Check technical track renewal ---
        if m == MAX_MONTHS:
            lifetime = t
            break

        if track_results:
            history.append({
                'Month':m, 'H_H':H_H,'RCF_H':R_H,
                'H_L':H_L,'RCF_L':R_L,'Gauge':gauge
            })

    # --- Compute annuity ---
    total_PV = PV_maint + PV_cap + PV_renew
    annuity = total_PV / TRACK_LEN / lifetime

    return (annuity, lifetime, history) if track_results else (annuity, lifetime)




# Example usage of the function
grinding_freq_low = 6  # months
grinding_freq_high = 10  # months
gauge_freq = 40  # months

annuity, lifetime, history = get_annuity_track(
    data_df,
    grinding_freq_low=grinding_freq_low,
    grinding_freq_high=grinding_freq_high,
    gauge_freq=gauge_freq,
    profile_low_rail=profile_low_rail,
    profile_high_rail=profile_high_rail,
    track_results=True
)

print(f"Annuity: {annuity:.2f} €/km/year")
print(f"Lifetime: {lifetime:.2f} years")

Annuity: 3716.92 €/km/year
Lifetime: 15.00 years


## Plots